In [1]:
import os, sys, email
import re
import numpy as np 
import pandas as pd


In [4]:
emails_df = pd.read_csv('/Users/peter/Library/CloudStorage/OneDrive-Personal/Documents/MDS_UBC/DATA_533/emails.csv')
print(emails_df.shape)
emails_df.head(5)

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
print(emails_df['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [38]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

import re

def strip_reply_chain(text):
    """
    Remove forwarded/reply chain segments.
    Keeps only the newest part of the email.
    """

    if not isinstance(text, str):
        return text

    # Common markers that indicate beginning of older messages
    patterns = [
        r"-----\s*Original Message\s*-----",
        r"-----\s*Forwarded Message\s*-----",
        r"__+Original Message__+",
        r"Forwarded",
        r"Original Message",
        r"On\s.+wrote:",          # "On Jan 12, John wrote:"
        r"From:\s.*\n", 
        r".*To:.*@.*$"
    ]

    # Combine patterns into one regex
    splitter = re.compile("|".join(patterns), flags=re.IGNORECASE | re.MULTILINE)

    # Find first occurrence
    match = splitter.search(text)
    if match:
        return text[:match.start()].strip()

    return text.strip()

In [34]:
s_email_df = emails_df.sample(10000, random_state=42)

In [35]:
messages = list(map(email.message_from_string, s_email_df['message']))
s_email_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    s_email_df[key] = [doc[key] for doc in messages]
# Parse content from emails
s_email_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
s_email_df['From'] = s_email_df['From'].map(split_email_addresses)
s_email_df['To'] = s_email_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
s_email_df['user'] = s_email_df['file'].map(lambda x:x.split('/')[0])
del messages

s_email_df.head(1)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
427616,shackleton-s/sent/1912.,<21013688.1075844564560.JavaMail.evans@thyme>,"Tue, 29 Aug 2000 01:26:00 -0700 (PDT)",(sara.shackleton@enron.com),(william.bradford@enron.com),Re: Credit Derivatives,1.0,text/plain; charset=us-ascii,7bit,Sara Shackleton,William S Bradford,,,\Sara_Shackleton_Dec2000_June2001_1\Notes Fold...,SHACKLETON-S,sshackle.nsf,Bill: Thanks for the info. I also spoke wit...,shackleton-s


In [115]:
subset_df = s_email_df[["From", "To", "Subject", "Date", "content"]]
subset_df["clean_content"] = subset_df["content"].apply(strip_reply_chain)
subset_df.head()



/var/folders/01/hxm6ksm17lg91kg9ct00ty580000gn/T/ipykernel_79561/3261034206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["clean_content"] = subset_df["content"].apply(strip_reply_chain)


,From,To,Subject,Date,content,clean_content
427616,(sara.shackleton@enron.com),(william.bradford@enron.com),Re: Credit Derivatives,"Tue, 29 Aug 2000 01:26:00 -0700 (PDT)",Bill: Thanks for the info. I also spoke wit...,Bill: Thanks for the info. I also spoke wit...
108773,(pat.clynes@enron.com),(aimee.lannou@enron.com),Meter #1591 Lamay Gaslift,"Mon, 24 Apr 2000 05:43:00 -0700 (PDT)","Aimee,\nPlease check meter #1591 Lamay gas lif...","Aimee,\nPlease check meter #1591 Lamay gas lif..."
355471,(knipe3@msn.com),"(wollam.erik@enron.com, corrier.brad@enron.com...",Re: man night again?,"Thu, 2 May 2002 04:54:27 -0700 (PDT)",GCCA Crawfish and rip-off raffle & over-priced...,GCCA Crawfish and rip-off raffle & over-priced...
457837,(kalmeida@caiso.com),(chris.stokley@enron.com),"Enron 480, 1480 charges","Wed, 8 Aug 2001 14:35:08 -0700 (PDT)","<<Keoni.zip>> Chris, per your request here ar...","<<Keoni.zip>> Chris, per your request here are..."
124910,(chris.germany@enron.com),(thomas.engel@enron.com),Transport Deal,"Wed, 21 Jun 2000 04:58:00 -0700 (PDT)",I'm trying to change the Receipt Meter on deal...,I'm trying to change the Receipt Meter on deal...


In [116]:
subset_df['Subject'] = subset_df['Subject'].astype(str).str.strip()
subset_df = subset_df[subset_df['Subject'] != ""]
subset_df = subset_df[subset_df['Subject'].str.upper() != "RE:"]

/var/folders/01/hxm6ksm17lg91kg9ct00ty580000gn/T/ipykernel_79561/2935866792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Subject'] = subset_df['Subject'].astype(str).str.strip()


In [117]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

texts = subset_df["Subject"].astype(str).tolist()

embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  
)


print(embeddings.shape)

Batches:   0%|          | 0/147 [00:00<?, ?it/s]

(9376, 384)


In [132]:
from sklearn.decomposition import PCA

# embeddings: (n_samples, d)
pca = PCA(n_components=50, random_state=42)
X = pca.fit_transform(embeddings)

import hdbscan
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=40,
    metric='euclidean'
)
labels = clusterer.fit_predict(X)

import numpy as np

# Count clusters (excluding noise = -1)
unique, counts = np.unique(labels[labels != -1], return_counts=True)

# Combine into list of (cluster_id, size)
cluster_sizes = list(zip(unique, counts))

# Sort by size descending
cluster_sizes_sorted = sorted(cluster_sizes, key=lambda x: -x[1])

# Print
print("Cluster sizes (largest → smallest):")
for cid, size in cluster_sizes_sorted:
    print(f"Cluster {cid}: {size} points")


/opt/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Cluster sizes (largest → smallest):
Cluster 3: 1731 points
Cluster 2: 244 points
Cluster 1: 98 points
Cluster 0: 42 points


In [133]:
from sklearn.metrics import silhouette_score


subset_df["cluster"] = labels


mask = labels != -1
if mask.sum() > 1 and len(set(labels[mask])) > 1:
    sil = silhouette_score(X[mask], labels[mask])
    print("Silhouette (no noise):", sil)
else:
    print("Not enough clustered points for silhouette.")

Silhouette (no noise): 0.2715246081352234


In [139]:
subjects = subset_df[subset_df['cluster'] ==3 ]['Subject']

for i, s in enumerate(subjects, 1):
    print(f"{i}. {s}")

1. Re: man night again?
2. Leona Tan
3. RE: Hello
4. Hey Baby.
5. Re: (no subject)
6. Sun Devil - Helicopter Fly-Over of Routes
7. Re: Thanks and a few residual questions
8. Info
9. Americans
10. RE: Do we play today?
11. EMAZING Positive Living Tip of the Day - Assert Yourself At Work
12. Fw: : First Time Golfer
13. Re: Sailing
14. Re: Greg -
15. FW: Friendship!
16. Sen. Dunn
17. Re: Oxy stuff
18. Re: REPLY TO: RRSP
19. this
20. Gallup
21. Re: SERC agreement
22. How Are You?
23. Greetings
24. Re: Meeting Nov 8th
25. Re: B-Day's
26. Chip Saltsman
27. Re: #516508 and 516509
28. DynegyDirect
29. Welcome Back
30. Re: Q90172.1
31. RE: language
32. PLEASE LOOK AT THIS
33. Name Overlay 04/19/01
34. Thanks from Charles
35. FW: Capacity release
36. Re: Redline doc
37. Re: wouldn't you like to see us
38. RE: Strongman Outline
39. RE: Thursday, October 25
40. re[2]: re[2]: Stoneville
41. Re: Pics [CGEY Virus checked]
42. Fwd: Fw: HEALTH CONCERN - PLEASE READ
43. Re: Update
44. RE: How are you ?


In [ ]:
#blind classification
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",   # or any NLI model
    device=0                            # -1 for CPU, 0 for first GPU
)

labels = ["Study", "Appointment", "Financial", "Business",
          "Commercial", "Fraud", "Trash", "Others"]

def classify_email_zero_shot(subject, body):
    text = f"Subject: {subject}\n\nBody: {body}"
    result = classifier(text, labels, multi_label=False)
    return result["labels"][0]   # top predicted label

# Example
row = subset_df.iloc[0]
pred = classify_email_zero_shot(row["Subject"], row["clean_content"])
print(pred)



In [ ]:
#classify with llm
def classify_email(subject, body):
    prompt = f"""Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: {subject}  
    Body: {body}  

    **Category:**"""
    
    device = "cuda:0"
    
  
    inputs = tokenizer(prompt, return_tensors="pt")
    

    max_input_tokens = 512
    input_length = inputs.input_ids.shape[1]

    if input_length > max_input_tokens:

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_input_tokens)


    inputs = inputs.to(device)


    output = model.generate(**inputs, max_new_tokens=50)

    return tokenizer.decode(output[0], skip_special_tokens=True)


subject = subset_df.iloc[0]["Subject"]
body = subset_df.iloc[0]["content"]
category = classify_email(subject, body)
print(f"Email: {subject}\nDự đoán: {category}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Email: Wind River - Conversion Checklist
Dự đoán: Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: Wind River - Conversion Checklist  
    Body: Attached please find an updated draft of the conversion closing checklist.

If you have any questions or if I can be of further assistance, please feel 
free to call me at (713) 220-4427.

S.B.


 - LostCree.DOC  

    **Category:** Commercial

    **Explanation:** This email is about a commercial transaction. It is not a personal email. The subject line mentions the name of a company and the body of the email contains a list of documents. The email is addressed to a person named


In [21]:
subject1 = subset_df.iloc[1]["Subject"]
body1 = subset_df.iloc[1]["content"]
category = classify_email(subject1, body1)
print(f"Email: {subject1}\nDự đoán: {category}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Email: RE: Austin Energy
Dự đoán: Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: RE: Austin Energy  
    Body: It may also makes sense to keep the arrangements separate.  The City's claim pursuant to its power deal (presumably with ENA) is likely a general unsecured claim that gives them very little hope of actually collecting.  Sandhill is not a debtor (not yet anyway) and is an assets/commodity that can be freely assigned.  Just a thought.

 -----Original Message-----
From: 	Mann, Kay  
Sent:	Wednesday, December 12, 2001 12:23 PM
To:	Zisman, Stuart
Subject:	FW: Austin Energy



 -----Original Message-----
From: 	Clark, Barton  
Sent:	Wednesday, December 12, 2001 11:42 AM
To:	Keenan, Jeffrey
Cc:	Mann, Kay
Subject:	RE: Austin Energy

For now, Kay and I both